# Домашнее задание №3
## 1	Загрузить из БД данные о параметрах заемщиках (из второго задания) и флаги {дефолт/недефолт} на горизонте 1 год. 

In [1]:
import sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#соединяемся с созданной в задании 1 базой данных data_base
conn = sqlite3.connect("data_base.db")
cursor = conn.cursor()

payments_table = pd.read_sql_query("SELECT * FROM payments", conn)

In [3]:
#изменяемая переменная горизонта риска соответсвует заданию (1 год = 12 месяцев)
risk_horizon = 12

In [4]:
#Создадим переменную, в которую будут добавляться данные типа (номер контракта, дата дефолта, флаг дефолт/недефолт)
default_risk_horizon = []

contract_number = list(payments_table['contract_number'])
contract_number_unique = list(set(contract_number))

for i in contract_number_unique:
    table = payments_table[payments_table['contract_number'] == i]
    if table.shape[0] >= risk_horizon:
        table = table[:risk_horizon]
        table = table.reset_index()
        for j in range(0, table.shape[0]-3):
            if (table['amount_paid'][j] < table['amount_due'][j] and table['amount_paid'][j+1] < table['amount_due'][j+1] and
                table['amount_paid'][j+2] < table['amount_due'][j+2] and table['amount_paid'][j+3] < table['amount_due'][j+3]):
                default_date = table['date'][j + 3]
                default_date = (str(pd.to_datetime(default_date).year) + '-' + str(pd.to_datetime(default_date).month) + 
                                '-' + str(pd.to_datetime(default_date).day))
                while (i, '0000-00-00', 0) in default_risk_horizon: #Здесь мы лезим в общий список просрочек и контрактов, просто фиктивные переменные 0 и 1 не подойдут
                                default_risk_horizon.remove ((i, '0000-00-00', 0)) #0000-00-00 для удобства обозначения год-месяц-день при отсутвии дефолта, далее дроп
                default_risk_horizon.append((i, default_date, 1)) 
                break 
            else: default_risk_horizon.append((i, '0000-00-00', 0))

In [5]:
#объединим контракты, чтобы на выходе таблица содержала в себе данные типа: номер контракта(уникальный),
#дата просрочки(или же 0000-00-00 если ее нет) и флаг дефолта(фиктивная перменная 0 или 1)
default_risk_horizon = list(set(default_risk_horizon))
print (len(default_risk_horizon))
default_risk_horizon

59


[(100441, '0000-00-00', 0),
 (100368, '0000-00-00', 0),
 (100610, '0000-00-00', 0),
 (100684, '0000-00-00', 0),
 (100703, '0000-00-00', 0),
 (100206, '0000-00-00', 0),
 (100520, '0000-00-00', 0),
 (100648, '0000-00-00', 0),
 (100122, '0000-00-00', 0),
 (100506, '0000-00-00', 0),
 (100068, '0000-00-00', 0),
 (100614, '0000-00-00', 0),
 (100933, '0000-00-00', 0),
 (100476, '0000-00-00', 0),
 (100022, '0000-00-00', 0),
 (100524, '0000-00-00', 0),
 (100689, '0000-00-00', 0),
 (100872, '0000-00-00', 0),
 (100493, '0000-00-00', 0),
 (100932, '0000-00-00', 0),
 (100582, '0000-00-00', 0),
 (100585, '0000-00-00', 0),
 (100847, '0000-00-00', 0),
 (100990, '0000-00-00', 0),
 (100737, '0000-00-00', 0),
 (100797, '0000-00-00', 0),
 (100468, '0000-00-00', 0),
 (100743, '0000-00-00', 0),
 (100176, '0000-00-00', 0),
 (100323, '0000-00-00', 0),
 (100620, '0000-00-00', 0),
 (100127, '0000-00-00', 0),
 (100516, '0000-00-00', 0),
 (101028, '0000-00-00', 0),
 (100297, '0000-00-00', 0),
 (100211, '0000-00-0

In [6]:
#соединяемся с созданной базой данных data_base
#создаем пустую заготовку в базе данных для размещения данных по дефолтам на горизоне риска
conn = sqlite3.connect("data_base.db")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE Risk_horizon12 (
contract_number INT,
default_date DATE,
default_flag BOOLEAN)""".format(risk_horizon))

conn.commit()

In [7]:
#заполняем таблицу готовыми данными из таблицы default_risk_horizon
cursor = conn.cursor()
for i in default_risk_horizon:
    cursor.execute("""INSERT INTO Risk_horizon12 
    VALUES
    (%d, '%s', %d)""".format(risk_horizon) % (
    i[0],
    i[1],
    i[2]))
    
conn.commit()

In [8]:
#по ходу написания кода было обнаружено, что контракт 100875 не имеет данных о платежах, поэтому считать его за "дефолт" мы не будем
#уберем данный контракт из получившейся таблицы Risk_horizon12 
conn = sqlite3.connect("data_base.db")
cursor = conn.cursor()

cursor.execute("DELETE FROM Risk_horizon12 WHERE contract_number = 100875")

conn.commit()
#делаем вывод, что на горизонте риска год нет дефолта